<h1 align="center"> Sensordatenfusion Tutorium 04 </h1>

<h3 align="center"> Messmodel und Updater </h3>

In [23]:
import scipy as sp
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.base import Property
from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

<h3 align="center"> Messmodell erstellen </h3>

In [ ]:
class SDFMessmodell(MeasurementModel, LinearModel, GaussianModel):
    
    @property
    def ndim_meas(self):
        return 2
    
    def matrix(self, **kwargs):
        # model_matrix = np.array([["""Erste Zeile der Messmatrix"""], [""" Zweite Zeile der Messmatrix"""]])
        model_matrix = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
        return model_matrix
    
    def covar(self):
        # return  np.array([["""Erste Zeile der Rauschmatrix R"""], ["""Zweite Zeile der Rauchmatrix R"""]])
        return np.array([[0.75, 0],[0, 0.75]])
    
    def rvs(self):
        # sample ziehen aus der Kovarianzmatrix
        noise = multivariate_normal.rvs(sp.zeros(self.ndim_meas), self.covar(), 1)
        return noise.reshape((-1, 1))
    
    def pdf(self):
        pass
    
    
    

In [25]:
messmodell = SDFMessmodell(4, (0, 2))

In [26]:
messmodell.matrix()

array([[1, 0, 0, 0],
       [0, 0, 1, 0]])

In [27]:
messmodell.covar()

array([[0.75, 0.  ],
       [0.  , 0.75]])

In [28]:
print(messmodell.rvs())

[[-0.49726351]
 [ 0.33421274]]


<h3 align="center"> Updater erstellen </h3>

In [29]:
from functools import lru_cache

from stonesoup.updater.base import Updater
from stonesoup.base import Property
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.prediction import GaussianMeasurementPrediction
from stonesoup.types.update import GaussianStateUpdate

In [ ]:
class SDFUpdater(Updater):
    messprediction = None   # mess-mean
    kalman_gain = None  # messkovarianz
    Pxy = None

    @lru_cache
    def get_measurement_prediction(self, state_prediction, measurement_model=None, **kwargs):
        measurement_matrix = measurement_model.matrix()
        measurement_noise_covar = measurement_model.covar()
        state_prediction_mean = state_prediction.mean
        state_prediction_covar = state_prediction.covar

        self.messprediction = measurement_matrix @ state_prediction_mean
        self.kalman_gain = measurement_matrix @ state_prediction_covar @ measurement_matrix.T + measurement_noise_covar
        self.Pxy = state_prediction_covar @ measurement_matrix.T

        return GaussianMeasurementPrediction(self.messprediction, self.kalman_gain,
                                             state_prediction.timestamp,
                                             self.Pxy)

    def update(self, hypothesis, measurementmodel, **kwargs):
        K = self.Pxy @ np.linalg.pinv(self.kalman_gain)
        x_post = self.messprediction + K @ (hypothesis.measurement.state_vector - hypothesis.prediction.mean)
        P_post = self.kalman_gain - K @ self.Pxy.T
        P_post = (P_post+P_post.T)/2
        
        posterior_mean = x_post
        posterior_covar = P_post
        meas_pred_mean = self.messprediction
        meas_pred_covar = self.kalman_gain
        cross_covar = self.Pxy
        _ = K
        
        # Augment hypothesis with measurement prediction
        hypothesis = SingleHypothesis(hypothesis.prediction,
                                      hypothesis.measurement,
                                      GaussianMeasurementPrediction(
                                          meas_pred_mean, meas_pred_covar,
                                          hypothesis.prediction.timestamp,
                                          cross_covar)
                                      )

        return GaussianStateUpdate(posterior_mean,
                                   posterior_covar,
                                   hypothesis,
                                   hypothesis.measurement.timestamp)